In [ ]:
import opendatasets as od
od.download(dataset_id_or_url="https://www.kaggle.com/datasets/aslanahmedov/number-plate-detection",data_dir="C:\\Users\\amith\\Documents\\Datasets\\9 - Automatic Number Plate Recognition using ResNet and CNN")


In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import xml.etree.ElementTree as et
import os
import cv2
import tensorflow as tf
from sklearn.metrics import *


In [2]:
img_list_path = "C:\\Users\\amith\\Documents\\Datasets\\9 - Automatic Number Plate Recognition using ResNet and CNN\\number-plate-detection\\images"


xml_list = list(map(lambda x: x if ".xml" in x else None, os.listdir(img_list_path)))

while None in xml_list:
    xml_list.remove(None)


img_list = list(map(lambda x: x[:x.index(".")] + ".jpeg",xml_list))
img_num = list(map(lambda x: int(x[x.index("N")+1:x.index(".")]),xml_list))

df = pd.DataFrame(columns=["Image List","XML List","Image Number"])
df["Image List"] = img_list
df["XML List"] = xml_list
df["Image Number"] = img_num

df = df.sort_values(by="Image Number").reset_index(drop=True)


In [3]:
df.head()


,Image List,XML List,Image Number
0,N1.jpeg,N1.xml,1
1,N2.jpeg,N2.xml,2
2,N3.jpeg,N3.xml,3
3,N4.jpeg,N4.xml,4
4,N5.jpeg,N5.xml,5


In [57]:
df_1 = pd.DataFrame(columns=["x_min","y_min","x_max","y_max"])

img = []
img_resized = []

for i in range(df.shape[0]):   
    img_full_path = img_list_path + "\\" + df.loc[i,"Image List"]
    xml_full_path = img_list_path + "\\" + df.loc[i,"XML List"]

    img_full = cv2.imread(img_full_path)
    img_resized_1 = cv2.resize(src=img_full,dsize=(128,128)) / 255.0
    img_rescaled = img_full / 255.0
    img_resized.append(img_resized_1)

    img.append(img_rescaled)

    xml_tree = et.parse(xml_full_path)
    xml_root = xml_tree.getroot()

    size = xml_root.find("size")

    height = int(size.find("height").text)
    width = int(size.find("width").text)

    object = xml_root.find("object")
    bndbox = object.find("bndbox")

    x_min = int(bndbox.find("xmin").text)
    y_min = int(bndbox.find("ymin").text)
    x_max = int(bndbox.find("xmax").text)
    y_max = int(bndbox.find("ymax").text)

    df_1.loc[i,"x_min"] = x_min / width
    df_1.loc[i,"y_min"] = y_min / height
    df_1.loc[i,"x_max"] = x_max / width
    df_1.loc[i,"y_max"] = y_max / height

img_resized = np.array(img_resized)


In [91]:
height,width


(486, 720)

In [92]:
img_full.shape


(486, 720, 3)

In [75]:
X = img_resized.copy()
y = np.array(df_1)

pos = np.array(list(range(y.shape[0])))

np.random.shuffle(pos)

val_test_threshold = int(pos.shape[0] * 0.8)
train_val_threshold = int(val_test_threshold * 0.8)

train_pos = pos[:train_val_threshold]
val_pos = pos[train_val_threshold:val_test_threshold]
test_pos = pos[val_test_threshold:]

X_train = X[train_pos].astype(np.float64)
X_val = X[val_pos].astype(np.float64)
X_test = X[test_pos].astype(np.float64)

y_train = y[train_pos].astype(np.float64)
y_val = y[val_pos].astype(np.float64)
y_test = y[test_pos].astype(np.float64)


In [78]:
model = tf.keras.models.Sequential(layers=[

    tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),activation="relu",input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=256,activation="relu"),
    tf.keras.layers.Dense(units=4,activation="sigmoid")
])


In [79]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.mean_absolute_error,metrics=["accuracy"])


In [80]:
model.fit(x=X_train,y=y_train,batch_size=10,epochs=100,verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=2),validation_data=(X_val,y_val))


Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.5158 - loss: 0.1379 - val_accuracy: 0.5000 - val_loss: 0.1178
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5079 - loss: 0.1224 - val_accuracy: 0.5000 - val_loss: 0.1150
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5798 - loss: 0.1156 - val_accuracy: 0.7222 - val_loss: 0.1133
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6691 - loss: 0.1132 - val_accuracy: 0.5000 - val_loss: 0.1106
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6638 - loss: 0.1042 - val_accuracy: 0.6389 - val_loss: 0.1073
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6773 - loss: 0.1100 - val_accuracy: 0.6389 - val_loss: 0.1000
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7017 - loss: 0.0959 - val_accuracy: 0.7778 - val_loss: 0.1011
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7533 - loss: 0.0914 - val_accuracy: 0.

In [82]:
model.evaluate(x=X_test,y=y_test)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7389 - loss: 0.0747


[0.07572047412395477, 0.7333333492279053]

In [95]:
y_pred = model.predict(X_test)
y_pred[0]


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  


array([0.3030235 , 0.36404812, 0.7125954 , 0.61149716], dtype=float32)

In [93]:
y_pred[0]

#    df_1.loc[i,"x_min"] = x_min / width
#    df_1.loc[i,"y_min"] = y_min / height
#    df_1.loc[i,"x_max"] = x_max / width
#    df_1.loc[i,"y_max"] = y_max / height


array([0.3030235 , 0.36404812, 0.7125954 , 0.61149716], dtype=float32)

In [87]:
root_mean_squared_error(y_test,y_pred)


0.10126615910504846

In [90]:
tf.keras.models.save_model(model=model,filepath="C:\\Users\\amith\\Documents\\GitHub\\udemy\\9 - Automatic Number Plate Recognition using ResNet and CNN\\app\\NPD.h5")


tf.keras.models.save_model(model=model,filepath="C:\\Users\\amith\\Documents\\GitHub\\udemy\\9 - Automatic Number Plate Recognition using ResNet and CNN\\ML_model\\NPD.h5")
